In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/itk-2020/x_test.csv
/kaggle/input/itk-2020/y_train.csv
/kaggle/input/itk-2020/x_train.csv
/kaggle/input/itk-2020/y_test_baseline.csv


In [2]:
import pandas as pd
import numpy as np
#K Fold CV K=10
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.model_selection import RandomizedSearchCV

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier

from sklearn import metrics

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV  #Perforing grid search



import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

## helper functions

In [3]:
def get_train_data():
    forestCoverX = pd.read_csv("/kaggle/input/itk-2020/x_train.csv")
    forestCoverY = pd.read_csv("/kaggle/input/itk-2020/y_train.csv")

    forestCoverX = forestCoverX.drop(['id'], axis=1)
    forestCoverY = forestCoverY["Cover_Type"]


    forestCoverX = forestCoverX.values
    forestCoverY = forestCoverY.values
    
    return [forestCoverX, forestCoverY]  

In [4]:
def check_model(model, cross_validation_method, forestCoverX, forestCoverY):
    

    if cross_validation_method == "holdout":
        
    # Split into train and test sets
        X_train, X_val, y_train, y_val= train_test_split(forestCoverX,
                                        forestCoverY,
                                        test_size=0.20,
                                        random_state=32)



        # fit the model with the training data
        model.fit(X_train,y_train)


        # predict the target on the train dataset
        predict_train = model.predict(X_train)

        # Accuray Score on train dataset
        accuracy_train = accuracy_score(y_train,predict_train)
        print('\n accuracy_score on train dataset : ', accuracy_train)

        # predict the target on the test dataset
        predict_val = model.predict(X_val)

        # Accuracy Score on test dataset
        accuracy_val = accuracy_score(y_val,predict_val)
        print('\naccuracy_score with hold_out validate dataset : ', accuracy_val)
    
    elif cross_validation_method == "k-fold":
        
        accuracies = cross_val_score(estimator = model, X=forestCoverX, y=forestCoverY, cv=5)
        print('\naccuracy_score with k-fold all accuracies : ', accuracies)
        print('\naccuracy_score with k-fold mean : ', accuracies.mean())
    
    return

In [5]:
def fit_model(model,forestCoverX, forestCoverY):
    
    model.fit(forestCoverX,forestCoverY)
    
    return model

In [6]:
def getSubmissionCSV(classifier, csv_name):
    forestCoverXSubmit = pd.read_csv("/kaggle/input/itk-2020/x_test.csv")
    forestCoverXSubmit_test = forestCoverXSubmit.drop(['id'], axis=1)
    
    forestCoverXSubmit_test = forestCoverXSubmit_test.values
    
    y_final = classifier.predict(forestCoverXSubmit_test)
    
    index_range = np.arange( forestCoverXSubmit_test.shape[0])
    data = {'id': index_range.tolist(), 'Cover_Type': y_final.tolist()}
    
    output_df = pd.DataFrame(data = data ) 
    output_df.to_csv(csv_name, index=False)

## Add euclidean distance

In [ ]:
# def getEuclidenAndCategoricalTraindata():
    
#     forestCoverX = pd.read_csv("/kaggle/input/itk-2020/x_train.csv")
    
#     vert_dist = forestCoverX['Vertical_Distance_To_Hydrology'].values
#     hor_dist = forestCoverX['Horizontal_Distance_To_Hydrology'].values

#     euc_distance = np.sqrt(vert_dist**2 + hor_dist**2)

#     forestCoverX['Euc_Distance_To_Hydrology'] = euc_distance
    
    
#     return forestCoverX

## Remove soil type

In [ ]:
# def get_train_data_remove_soil_types():
#     forestCoverX = pd.read_csv("/kaggle/input/itk-2020/x_train.csv")
#     forestCoverY = pd.read_csv("/kaggle/input/itk-2020/y_train.csv")
    
#     drop_columns = ['id' ,'Soil_Type_1', 'Soil_Type_2', 'Soil_Type_3',
#        'Soil_Type_4', 'Soil_Type_5', 'Soil_Type_6', 'Soil_Type_7',
#        'Soil_Type_8', 'Soil_Type_9', 'Soil_Type_10', 'Soil_Type_11',
#        'Soil_Type_12', 'Soil_Type_13', 'Soil_Type_14', 'Soil_Type_15',
#        'Soil_Type_16', 'Soil_Type_17', 'Soil_Type_18', 'Soil_Type_19',
#        'Soil_Type_20', 'Soil_Type_21', 'Soil_Type_22', 'Soil_Type_23',
#        'Soil_Type_24', 'Soil_Type_25', 'Soil_Type_26', 'Soil_Type_27',
#        'Soil_Type_28', 'Soil_Type_29', 'Soil_Type_30', 'Soil_Type_31',
#        'Soil_Type_32', 'Soil_Type_33', 'Soil_Type_34', 'Soil_Type_35',
#        'Soil_Type_36', 'Soil_Type_37', 'Soil_Type_38', 'Soil_Type_39',
#        'Soil_Type_40']

#     forestCoverX = forestCoverX.drop(drop_columns, axis=1)
#     forestCoverY = forestCoverY["Cover_Type"]


#     forestCoverX = forestCoverX.values
#     forestCoverY = forestCoverY.values
    
#     return [forestCoverX, forestCoverY]  

..................................................................................

## KNN

In [ ]:
# forestCoverX = getEuclidenAndCategoricalTraindata()

In [28]:
from sklearn.neighbors import KNeighborsClassifier

In [29]:
forestCoverX, forestCoverY = get_train_data()

knn_classifier = KNeighborsClassifier()

check_model(knn_classifier, "holdout", forestCoverX, forestCoverY)

#fit knn_classifier
knn_classifier.fit(forestCoverX, forestCoverY)


 accuracy_score on train dataset :  0.9797243719380636

accuracy_score with hold_out validate dataset :  0.9613975559981314


KNeighborsClassifier()

## multi class logistic regression

In [ ]:
# forestCoverX = getEuclidenAndCategoricalTraindata()

In [74]:
from sklearn.linear_model import LogisticRegression

forestCoverX, forestCoverY = get_train_data()

lm = LogisticRegression()

check_model(lm, "holdout", forestCoverX, forestCoverY)

# lm.fit(forestCoverX, forestCoverY)


 accuracy_score on train dataset :  0.6201846535901109

accuracy_score with hold_out validate dataset :  0.6189914189471614


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


## Decision tree classifier

In [76]:
from sklearn.tree import DecisionTreeClassifier

forestCoverX, forestCoverY = get_train_data()

dt_classifer = DecisionTreeClassifier()

check_model(dt_classifer, "holdout", forestCoverX, forestCoverY)


 accuracy_score on train dataset :  1.0

accuracy_score with hold_out validate dataset :  0.9298640308819552


# Random Forest

In [40]:
forestCoverX, forestCoverY  =  get_train_data()

In [ ]:
# forestCoverX = getEuclidenAndCategoricalTraindata()

In [41]:
from sklearn.ensemble import RandomForestClassifier

In [42]:
rf_classifier = RandomForestClassifier()

In [43]:
cv_method = "holdout"
check_model(rf_classifier, cv_method, forestCoverX, forestCoverY)


 accuracy_score on train dataset :  1.0

accuracy_score with hold_out validate dataset :  0.9466081483120651


## with k fold cross validation

In [ ]:
rf_classifier_k_fold = RandomForestClassifier()

In [ ]:
cv_method = "k-fold"
check_model(rf_classifier_k_fold, cv_method, forestCoverX, forestCoverY)

### Get a solution

In [ ]:
rf_classier =  fit_model(rf_classifier_k_fold,forestCoverX, forestCoverY)

solution_name = 'y_test_solution_simple_rf_classifer.csv'
getSubmissionCSV(rf_classier, solution_name)

## with parameter check with k-fold validation. 

Get random grid

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 20)]

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 100, num = 20)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 3, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 3, 4]


# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              }

use random_search to get rough area. 

In [ ]:
def get_samples_for_random_search(sample_size = 50000):
    row_length = forestCoverX.shape[0]

    idx = np.random.randint(row_length, size=sample_size)

    X_samples =  forestCoverX[idx,:]
    y_samples =  forestCoverY[idx]
    
    return [X_samples, y_samples]

In [ ]:
rf = RandomForestClassifier()

X_samples, y_samples = get_samples_for_random_search()

#use 10% of the data, as 400,000 data would take forever, I would use 50,000 randomly selected rows


# search across 20 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions 
                               = random_grid, n_iter = 20, cv = 3, 
                               verbose=2, random_state=42, n_jobs = -1)


rf_random.fit(X_samples, y_samples)

In [ ]:
rf_random.best_params_

With this, we narrow the search area down

### Grid Search with cross validation on all data

In [ ]:
# param_grid  = { 'n_estimators': [2000],
#                 'max_depth': [50, 60],
#                'min_samples_split': [2,3,4],
#                'min_samples_leaf': [1],
#               }

# rf = RandomForestClassifier()

# grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
#                           cv = 3, verbose = 2 , n_jobs = -1)

In [ ]:
# grid_search.fit(forestCoverX, forestCoverY)

In [44]:
best_rf =  RandomForestClassifier(n_estimators= 1000,
             max_depth= 50,
             min_samples_split= 3,
                min_samples_leaf= 1 )

In [ ]:
# forestCoverX = forestCoverX.astype(np.int32)
# forestCoverY = forestCoverY.astype(np.int32)

In [45]:
cv_method = "holdout"
check_model(best_rf, cv_method, forestCoverX, forestCoverY)


 accuracy_score on train dataset :  0.999996926538114

accuracy_score with hold_out validate dataset :  0.9470015490152689


## Provide solution and submit

In [ ]:
# forestCoverX, forestCoverY = get_train_data()

In [ ]:
rf_classifer =  fit_model(best_rf,forestCoverX, forestCoverY)

solution_name = 'y_test_solution_1000_rf_classifer.csv'
getSubmissionCSV(rf_classifer, solution_name)

## Get min_max scaled values

In [17]:
forestCoverX = pd.read_csv("/kaggle/input/itk-2020/x_train.csv")
forestCoverY = pd.read_csv("/kaggle/input/itk-2020/y_train.csv")

forestCoverX = forestCoverX.drop(['id'], axis=1)
forestCoverY = forestCoverY['Cover_Type']

from sklearn.model_selection import train_test_split

#convert to numpy
forestCoverX = forestCoverX.values
forestCoverY = forestCoverY.values


# Split into train and test sets
X_train, X_val, y_train, y_val= train_test_split(forestCoverX,
                                forestCoverY,
                                test_size=0.20,
                                random_state=42)

from sklearn.preprocessing import MinMaxScaler

# Instantiate MinMaxScaler and use it to rescale X_train and X_vl
scaler = MinMaxScaler(feature_range=(0, 1))
rescaledX_train = scaler.fit_transform(X_train)
rescaledX_val = scaler.fit_transform(X_val)

## Naive bayes

In [30]:
from sklearn.naive_bayes import GaussianNB

# Instantiate a RF classifier
nb_classifier = GaussianNB()

check_model(nb_classifier, "holdout", forestCoverX, forestCoverY)


 accuracy_score on train dataset :  0.4605090882267969

accuracy_score with hold_out validate dataset :  0.45770942440559614


## SVM

In [18]:
# from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

forestCoverX, forestCoverY = get_train_data()

# Instantiate a multi class SVM. 
# svm_classifier = OneVsRestClassifier(SVC())
svm_classifier = LinearSVC()

svm_classifier.fit(rescaledX_train, y_train)


print("Accuracy of random forest classifier Train : ", svm_classifier.score(rescaledX_train, y_train))

print("Accuracy of random forest classifier validation : ", svm_classifier.score(rescaledX_val, y_val))


Accuracy of random forest classifier Train :  0.7128095744484673
Accuracy of random forest classifier validation :  0.7130141870128592


## Bagging with decision trees

In [ ]:
forestCoverX, forestCoverY = get_train_data()

In [ ]:
# dtc = DecisionTreeClassifier(criterion="entropy")
# dtc = DecisionTreeClassifier()

bag_model=BaggingClassifier(base_estimator=dtc, n_estimators=100, bootstrap=True)

check_model(bag_model, "holdout", forestCoverX, forestCoverY)

# bag_model.fit(forestCoverX,forestCoverY)

In [ ]:
cv_method = "holdout"
check_model(bag_model, cv_method, forestCoverX, forestCoverY)

In [ ]:
solution_name = 'y_test_solution_1000_bagdtc_classifer.csv'
getSubmissionCSV(bag_model, solution_name)

## Gradient boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

forestCoverX, forestCoverY = get_train_data()

gradBoostModel = GradientBoostingClassifier ()

check_model(gradBoostModel,"holdout", forestCoverX, forestCoverY)

## xgboost

In [7]:
forestCoverX, forestCoverY = get_train_data()

In [8]:
def check_model_xgb(model, cross_validation_method, forestCoverX, forestCoverY):


    if cross_validation_method == "holdout":
        
    # Split into train and test sets
        X_train, X_val, y_train, y_val= train_test_split(forestCoverX,
                                        forestCoverY,
                                        test_size=0.20,
                                        random_state=32)



        # fit the model with the training data
        model.fit(X_train,y_train)


        # predict the target on the train dataset
        predict_train = model.predict(X_train)
        predict_train = [round(value) for value in predict_train]

        # Accuray Score on train dataset
        accuracy_train = accuracy_score(y_train,predict_train)
        print('\n accuracy_score on train dataset : ', accuracy_train)

        # predict the target on the test dataset
        predict_val = model.predict(X_val)
        predict_val = [round(value) for value in predict_val]

        # Accuracy Score on test dataset
        accuracy_val = accuracy_score(y_val,predict_val)
        print('\naccuracy_score with hold_out validate dataset : ', accuracy_val)
    
    elif cross_validation_method == "k-fold":
        
        accuracies = cross_val_score(estimator = model, X=forestCoverX, y=forestCoverY, cv=5)
        print('\naccuracy_score with k-fold all accuracies : ', accuracies)
        print('\naccuracy_score with k-fold mean : ', accuracies.mean())
    
    return

In [9]:
# fit model no training data
xgb_model = XGBClassifier(n_estimators = 200 , learning_rate = 0.3, max_depth=20, n_jobs = -1)

In [13]:
check_model_xgb(xgb_model, "holdout", forestCoverX, forestCoverY)

In [ ]:
xgb_model.fit(forestCoverX,forestCoverY)

In [ ]:
def getSubmissionCSV_XGB(classifier, csv_name):
    forestCoverXSubmit = pd.read_csv("/kaggle/input/itk-2020/x_test.csv")
    forestCoverXSubmit_test = forestCoverXSubmit.drop(['id'], axis=1)
    
    forestCoverXSubmit_test = forestCoverXSubmit_test.values
    
    y_final = classifier.predict(forestCoverXSubmit_test)
    y_final = [round(value) for value in y_final]
    
    index_range = np.arange( forestCoverXSubmit_test.shape[0])
    data = {'id': index_range.tolist(), 'Cover_Type': y_final}
    
    output_df = pd.DataFrame(data = data ) 
    output_df.to_csv(csv_name, index=False)

In [ ]:
solution_name = 'y_test_solution_xgb_max_depth_8.csv'
getSubmissionCSV_XGB(xgb_model, solution_name)

## without soil types get worse results

In [ ]:
# forestCoverX, forestCoverY = get_train_data_remove_soil_types()

In [ ]:
# xgb_model = XGBClassifier(n_estimators = 200 , learning_rate = 0.3, max_depth=7)

# xgb_model.fit(forestCoverX,forestCoverY)

In [12]:
# check_model_xgb(xgb_model, "holdout", forestCoverX, forestCoverY)

## Stacking 

### stacking with KNN, bagging classifier, xgboost, logistic regression

In [31]:
# get a stacking ensemble of models
def get_stacking():
    # define the base models
    level0 = list()
    level0.append(('knn', KNeighborsClassifier()))
    level0.append( ('bag' , BaggingClassifier(base_estimator=DecisionTreeClassifier(criterion="entropy")
                                                  , n_estimators=100, bootstrap=True) )  )
    level0.append(('xgb', XGBClassifier(n_estimators = 200 , learning_rate = 0.3, max_depth=20)))
   
    level1 = LogisticRegression()
    # define the stacking ensemble
    model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
    return model

In [ ]:
forestCoverX, forestCoverY = get_train_data()

stack_model = get_stacking()
check_model(stack_model, "holdout", forestCoverX, forestCoverY)

In [ ]:
solution_name = 'y_test_solution_stack_classifer.csv'
getSubmissionCSV(stack_model, solution_name)